# Tracking de jugadores de fútbol utilizando YOLOV5

## Resumen

Vamos a utilizar los modelos YOLOv5, los cuales fueron entrenados en el dataset COCO, el cuál contiene dos clases similares a las que buscamos detectar y en teoría debería ser capaz de reconocerlos, estas 2 clases son **person=persona** y **sports ball = pelota**.

Para realizar la experimentación se utilizan clips de videos de partidos de fútbol de la competencia de Kaggle [DFL - Bundesliga Data Shootout](https://www.kaggle.com/competitions/dfl-bundesliga-data-shootout), y sobre ellos se aplicó los modelos preentrenados en el dataset de COCO. Se probaron varias configuraciones de `iou_threshold`, ``confidence_threshold`` y ``input_resolution `` pero el resultado obtenido no fue el esperado.

Con estos modelos YOLOv5 preentrenados, no se logra detectar en la mayoría de los casaos la pelota de fútbol (Objeto pequeño y para terminos de trackíng se mueve rápido). Además el modelo detecta muchos objetos redudantes que no forman parte de este estudio, objetos como entrenadores, aficionados, utileros, camarógrafos, balones fuera del campo, recogepelotas, y otros objetos que no son de interés.

Finalmente, a pesar que el dataset COCO contiene las clases que nos interesa, los videos utilizados para las inferencias deben haber sido significativamente diferentes de las imágenes utilizadas para el entrenamiento (YOLOv5), por lo que es recomendable utilizar Transfer Learning y entrenar en un dataset propio.

# Conect to drive

In [1]:
# Conectandose a Google Drive para usarlo como directorio de trabajo
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# Montando el directorio de trabajo.
%cd /content/drive/MyDrive/Maestria/Tesis/TrackFootballPlayers/PreTrainedYOLOV5

/content/drive/MyDrive/Maestria/Tesis/TrackFootballPlayers/PreTrainedYOLOV5


In [4]:
!ls -l

total 684
-rw------- 1 root root 699766 Mar  2 01:59 TrackingJuadoresFutbolPreTraindeYOLOV5.ipynb


# Configuraciones

In [5]:
# Caraterísticas de la MV
!nvidia-smi

Thu Mar  2 02:00:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
# Declaración de variables
import os
HOME = os.getcwd()
print(HOME)

/content/drive/MyDrive/Maestria/Tesis/TrackFootballPlayers/PreTrainedYOLOV5


# Descargar data

Utilizaremos 20 clips de video de la competencia de Kaggle DFL - Bundesliga Data Shootout from Kaggle. 

In [7]:
!pip install kaggle --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 KB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [8]:
import os
#from getpass import getpass     
import json

In [9]:
f = open('/content/drive/MyDrive/Kaggle/kaggle.json')
cred = json.load(f)

os.environ['KAGGLE_USERNAME'] = cred['username']
os.environ['KAGGLE_KEY'] = cred['key']

In [10]:
!kaggle competitions files -c dfl-bundesliga-data-shootout | grep clips | head -10

clips/0a2d9b_7.mp4      19MB  2022-07-29 14:23:09  
clips/08fd33_2.mp4      21MB  2022-07-29 14:23:09  
clips/08fd33_3.mp4      17MB  2022-07-29 14:23:09  
clips/08fd33_0.mp4      19MB  2022-07-29 14:23:09  
clips/0a2d9b_2.mp4      20MB  2022-07-29 14:23:09  
clips/0a2d9b_0.mp4      20MB  2022-07-29 14:23:09  
clips/08fd33_4.mp4      19MB  2022-07-29 14:23:09  
clips/0a2d9b_9.mp4      18MB  2022-07-29 14:23:09  
clips/08fd33_8.mp4      20MB  2022-07-29 14:23:09  
clips/08fd33_6.mp4      19MB  2022-07-29 14:23:09  


In [11]:
%cd {HOME}
!kaggle competitions files -c dfl-bundesliga-data-shootout | \
grep clips | head -20 | \
awk '{print $1}' | \
while read -r line; \
    do kaggle competitions download -c dfl-bundesliga-data-shootout -f $line -p clips --quiet; \
    unzip ${line}.zip -d clips && rm ${line}.zip; \
    done

/content/drive/MyDrive/Maestria/Tesis/TrackFootballPlayers/PreTrainedYOLOV5
Archive:  clips/08fd33_8.mp4.zip
  inflating: clips/08fd33_8.mp4      
Archive:  clips/0a2d9b_4.mp4.zip
  inflating: clips/0a2d9b_4.mp4      
Archive:  clips/0a2d9b_3.mp4.zip
  inflating: clips/0a2d9b_3.mp4      
Archive:  clips/08fd33_1.mp4.zip
  inflating: clips/08fd33_1.mp4      
Archive:  clips/0a2d9b_0.mp4.zip
  inflating: clips/0a2d9b_0.mp4      
Archive:  clips/0a2d9b_5.mp4.zip
  inflating: clips/0a2d9b_5.mp4      
Archive:  clips/08fd33_7.mp4.zip
  inflating: clips/08fd33_7.mp4      
Archive:  clips/0a2d9b_6.mp4.zip
  inflating: clips/0a2d9b_6.mp4      
Archive:  clips/08fd33_2.mp4.zip
  inflating: clips/08fd33_2.mp4      
Archive:  clips/08fd33_0.mp4.zip
  inflating: clips/08fd33_0.mp4      
Archive:  clips/0a2d9b_2.mp4.zip
  inflating: clips/0a2d9b_2.mp4      
Archive:  clips/08fd33_6.mp4.zip
  inflating: clips/08fd33_6.mp4      
Archive:  clips/0a2d9b_9.mp4.zip
  inflating: clips/0a2d9b_9.mp4      
A

# Descargar e Instalar YOLOV5

In [12]:
%cd {HOME}
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt

import torch
import utils
display = utils.notebook_init()

YOLOv5 🚀 v7.0-116-g5c91dae Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.2/166.8 GB disk)


# Utilizar el modelo preentrenado de YOLOV5 en COCO

In [18]:
%%time
%cd {HOME}/yolov5
!python detect.py --weights yolov5x.pt --img 640 --conf 0.25 --source {HOME}/clips/08fd33_4.mp4 --name coco

/content/drive/MyDrive/Maestria/Tesis/TrackFootballPlayers/PreTrainedYOLOV5/yolov5
detect: weights=['yolov5x.pt'], source=/content/drive/MyDrive/Maestria/Tesis/TrackFootballPlayers/PreTrainedYOLOV5/clips/08fd33_4.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=coco, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-116-g5c91dae Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
video 1/1 (1/750) /content/drive/MyDrive/Maestria/Tesis/TrackFootballPlayers/PreTrainedYOLOV5/clips/08fd33_4.mp4: 384x640 25 persons, 1 sports ball, 58.6ms
video 1/1 (2/750) /content/drive/MyDrive/Maestria/Tesis/Track

In [19]:
%%time
%cd {HOME}/yolov5
!python detect.py --weights yolov5x6.pt --img 1280 --conf 0.25 --source {HOME}/clips/08fd33_4.mp4 --name coco2

/content/drive/MyDrive/Maestria/Tesis/TrackFootballPlayers/PreTrainedYOLOV5/yolov5
detect: weights=['yolov5x6.pt'], source=/content/drive/MyDrive/Maestria/Tesis/TrackFootballPlayers/PreTrainedYOLOV5/clips/08fd33_4.mp4, data=data/coco128.yaml, imgsz=[1280, 1280], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=coco2, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-116-g5c91dae Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5x6 summary: 574 layers, 140730220 parameters, 0 gradients
video 1/1 (1/750) /content/drive/MyDrive/Maestria/Tesis/TrackFootballPlayers/PreTrainedYOLOV5/clips/08fd33_4.mp4: 768x1280 27 persons, 1 sports ball, 133.4ms
video 1/1 (2/750) /content/drive/MyDrive/Maestria/Tes